In [2]:
import numpy as np
import pandas as pd

1) Даны значения величины заработной платы заемщиков банка (zp) 
и значения их поведенческого кредитного скоринга (ks):

    zp = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110],
    ks = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832].

Используя математические операции, посчитать коэффициенты линейной регрессии,
приняв за X заработную плату (то есть, zp - признак), 
а за y - значения скорингового балла (то есть, ks - целевая переменная).
Произвести расчет как с использованием intercept, так и без.

Коэффициенты уравнения линейной регрессии можно найти следующим образом:

$$b = \frac{\overline{yx} - \overline{y} \cdot {\overline{x}}}{\overline{x^2} - (\overline{x})^2};\\
a = \overline{y} - b \cdot {\overline{x}}.$$

In [12]:
zp = np.array([35, 45, 190, 200, 40, 70, 54, 150, 120, 110])
ks = np.array([401, 574, 874, 919, 459, 739, 653, 902, 746, 832])
zp_ks = ks*zp
b = (zp_ks.mean() - ks.mean()*zp.mean())/((zp**2).mean()-(zp.mean())**2) 
a = ks.mean() - b* zp.mean()
df = pd.DataFrame({'X(zp)': zp, 'Y(ks)': ks, 'a': a, 'b': b}, columns=['X(zp)', 'Y(ks)', 'a','b'])

print(df, f'y = {a} + {b}x', sep='\n\n')

   X(zp)  Y(ks)           a         b
0     35    401  444.177357  2.620539
1     45    574  444.177357  2.620539
2    190    874  444.177357  2.620539
3    200    919  444.177357  2.620539
4     40    459  444.177357  2.620539
5     70    739  444.177357  2.620539
6     54    653  444.177357  2.620539
7    150    902  444.177357  2.620539
8    120    746  444.177357  2.620539
9    110    832  444.177357  2.620539

y = 444.1773573243596 + 2.620538882402765x


2 )Посчитать коэффициент линейной регрессии при заработной плате (zp),
используя градиентный спуск (без intercept).


3) В каких случаях для вычисления доверительных интервалов и проверки статистических гипотез используется таблица значений функции Лапласа, а в каких - таблица критических точек распределения Стьюдента?



4 ) Произвести вычисления как в пункте 2, но с вычислением intercept. 
Учесть, что изменение коэффициентов должно производиться на каждом шаге одновременно

(то есть изменение одного коэффициента не должно влиять на изменение другого во время одной итерации).